# About

The process of generating trip logs is not very fast. At this point, it's worth making performance improvements in it.

In [4]:
import sys; sys.path.append("../src/")
from processing import parse_feeds_into_trip_logs, mta_archival_time_to_unix_timestamp
from processing import fetch_archival_gtfs_realtime_data

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
gtfs_r = dict()

for n in range(0, 60, 5):
    print(n + 1)
    gtfs_r[n] = fetch_archival_gtfs_realtime_data(kind='gtfs', timestamp='2014-09-18-09-' + str(1 + n).zfill(2))
    
print("Done!")

1
6
11
16
21
26
31
36
41
46
51
56
Done!


I have a pretty good idea of what the problem is, it's the `append` ops in `parse_message_into_action_log`. Let's see for sure though.

In [18]:
from processing import parse_message_into_action_log

In [22]:
parse_message_into_action_log(gtfs_r[0].entity[0], gtfs_r[0].entity[1], 
                              mta_archival_time_to_unix_timestamp('2014-09-18-09-01'))

,action,information_time,route_id,stop_id,time_assigned,trip_id
0,STOPPED_AT,1411045260,1,137S,1411044718,047600_1..S02R
1,EXPECTED_TO_ARRIVE_AT,1411045260,1,138S,1411044928,047600_1..S02R
2,EXPECTED_TO_DEPART_AT,1411045260,1,138S,1411044928,047600_1..S02R
3,EXPECTED_TO_ARRIVE_AT,1411045260,1,139S,1411045018,047600_1..S02R
4,EXPECTED_TO_DEPART_AT,1411045260,1,139S,1411045078,047600_1..S02R
5,EXPECTED_TO_ARRIVE_AT,1411045260,1,140S,1411045228,047600_1..S02R


In [8]:
%prun ret2 = parse_feeds_into_trip_logs([gtfs_r[0], gtfs_r[5]], [mta_archival_time_to_unix_timestamp('2014-09-18-09-01'), mta_archival_time_to_unix_timestamp('2014-09-18-09-06')])

Output (top only):

```
         105427380 function calls (104388274 primitive calls) in 93.838 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   314211    5.505    0.000    5.505    0.000 {pandas.lib.maybe_convert_objects}
 22341043    4.891    0.000    6.975    0.000 {built-in method isinstance}
   159620    2.781    0.000    6.753    0.000 internals.py:2841(_rebuild_blknos_and_blklocs)
  4616632    2.107    0.000    2.108    0.000 {built-in method hasattr}
198754/162270    2.070    0.000    9.529    0.000 base.py:142(__new__)
   348206    2.059    0.000    2.206    0.000 {pandas.lib.infer_dtype}
    93293    2.055    0.000    2.055    0.000 {pandas.lib.clean_index_list}
7744350/6741751    2.035    0.000    3.035    0.000 {built-in method len}
  2045438    2.009    0.000    3.124    0.000 common.py:357(_get_dtype_type)
  2452279    1.837    0.000    3.856    0.000 dtypes.py:74(is_dtype)
  1439446    1.771    0.000    1.772    0.000 {built-in method array}
   481623    1.739    0.000   10.102    0.000 internals.py:2649(make_block)
  2929009    1.685    0.000    1.701    0.000 {built-in method getattr}
   257646    1.404    0.000    2.565    0.000 numeric.py:607(require)
   272939    1.259    0.000    1.259    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    30947    1.117    0.000   14.548    0.000 internals.py:1827(convert)
    65844    1.057    0.000    6.671    0.000 internals.py:4830(get_empty_dtype_and_na)
  5241436    0.987    0.000    0.987    0.000 {built-in method issubclass}
   472952    0.951    0.000    0.951    0.000 {built-in method empty}
    62997    0.935    0.000    2.027    0.000 internals.py:4935(get_mgr_concatenation_plan)
   481623    0.930    0.000    5.129    0.000 internals.py:96(__init__)
    31193    0.834    0.000   89.156    0.003 frame.py:4346(append)
   479352    0.797    0.000    2.356    0.000 internals.py:2786(shape)
   317715    0.778    0.000    4.178    0.000 internals.py:1811(__init__)
   482273    0.773    0.000    3.447    0.000 internals.py:216(mgr_locs)
  2038008    0.766    0.000    2.084    0.000 generic.py:7(_check)
    93943    0.685    0.000    0.986    0.000 {method 'get_indexer' of 'pandas.index.IndexEngine' objects}
   198429    0.673    0.000    1.970    0.000 common.py:365(_asarray_tuplesafe)
   191723    0.641    0.000    6.961    0.000 cast.py:604(_soft_convert_objects)
   194687    0.636    0.000    0.636    0.000 {built-in method concatenate}
   189641    0.628    0.000    2.917    0.000 missing.py:245(array_equivalent)
    30907    0.626    0.000    0.875    0.000 arraysetops.py:305(in1d)
   225995    0.594    0.000    1.031    0.000 base.py:334(_simple_new)
  1052682    0.594    0.000    2.802    0.000 common.py:54(is_object_dtype)
   928272    0.584    0.000    0.584    0.000 {method 'ravel' of 'numpy.ndarray' objects}
    30582    0.574    0.000    0.574    0.000 {pandas.lib.fast_multiget}
    61854    0.564    0.000   25.995    0.000 internals.py:2978(apply)
   131493    0.558    0.000    4.184    0.000 internals.py:5141(is_null)
    97362    0.547    0.000    0.782    0.000 internals.py:5014(combine_concat_plans)
    64416    0.537    0.000    3.829    0.000 internals.py:4528(_merge_blocks)
   312619    0.525    0.000    5.791    0.000 base.py:1643(equals)
   456350    0.521    0.000    0.521    0.000 {built-in method arange}
  1172409    0.514    0.000    2.707    0.000 common.py:102(is_categorical_dtype)
   773813    0.512    0.000    1.331    0.000 numeric.py:414(asarray)
   127412    0.502    0.000    8.111    0.000 internals.py:2745(__init__)
    63361    0.499    0.000    7.259    0.000 series.py:135(__init__)
  1193619    0.499    0.000    0.499    0.000 {method 'view' of 'numpy.ndarray' objects}
   549818    0.493    0.000    0.679    0.000 range.py:458(__len__)
   102875    0.487    0.000    2.211    0.000 missing.py:120(_isnull_ndarraylike)
    63974    0.476    0.000    0.476    0.000 {pandas.lib.isnullobj}
   607996    0.474    0.000    7.543    0.000 base.py:3637(_ensure_index)
    61489    0.470    0.000   18.171    0.000 generic.py:2224(reindex)
    30907    0.457    0.000    4.555    0.000 base.py:1961(difference)
   357307    0.452    0.000    1.165    0.000 dtypes.py:347(is_dtype)
   126151    0.447    0.000    8.979    0.000 frame.py:252(__init__)
    93943    0.446    0.000    3.256    0.000 base.py:2217(get_indexer)
   358869    0.441    0.000    0.508    0.000 internals.py:301(ftype)
   186833    0.439    0.000    0.439    0.000 {built-in method array_equivalent_object}
   667758    0.435    0.000    0.689    0.000 common.py:333(_get_dtype)
  1070195    0.433    0.000    0.882    0.000 base.py:522(values)
    93579    0.428    0.000    0.746    0.000 base.py:1097(is_unique)
    31518    0.413    0.000    4.333    0.000 merge.py:1460(__init__)
   189512    0.404    0.000    0.404    0.000 generic.py:114(__init__)
    61489    0.388    0.000    7.687    0.000 generic.py:2350(_reindex_with_indexers)
   922563    0.383    0.000    1.426    0.000 common.py:89(is_datetime64tz_dtype)
   478002    0.370    0.000    0.370    0.000 {method 'reshape' of 'numpy.ndarray' objects}
    30582    0.367    0.000    4.912    0.000 base.py:1822(union)
   718181    0.349    0.000    2.672    0.000 common.py:69(is_datetimetz)
    70771    0.345    0.000    1.232    0.000 cast.py:672(_possibly_infer_to_datetimelike)
      611    0.343    0.001   69.490    0.114 processing.py:100(parse_message_into_action_log)
    66585    0.343    0.000    2.088    0.000 concat.py:24(get_dtype_kinds)
   191709    0.332    0.000    0.589    0.000 generic.py:2746(__setattr__)
  1675484    0.324    0.000    0.324    0.000 internals.py:179(mgr_locs)
  1407474    0.315    0.000    1.558    0.000 internals.py:2788(<genexpr>)
    32208    0.315    0.000    4.778    0.000 internals.py:4511(_consolidate)
    31518    0.313    0.000   19.159    0.001 merge.py:1599(get_result)
    93943    0.293    0.000    1.256    0.000 base.py:2390(_possibly_promote)
    91135    0.292    0.000    0.461    0.000 range.py:124(_simple_new)
   197378    0.291    0.000    0.491    0.000 shape_base.py:61(atleast_2d)
   127737    0.282    0.000    0.990    0.000 internals.py:3257(_consolidate_check)
   939563    0.281    0.000    0.365    0.000 base.py:491(__len__)
   124279    0.279    0.000    0.461    0.000 {built-in method sorted}
   836525    0.275    0.000    0.921    0.000 <frozen importlib._bootstrap>:2264(_handle_fromlist)
   515292    0.268    0.000    0.320    0.000 numeric.py:676(<genexpr>)
   [...]
```

And then:

In [23]:
%prun ret2 = parse_feeds_into_trip_logs([gtfs_r[0], gtfs_r[5]], [mta_archival_time_to_unix_timestamp('2014-09-18-09-01'), mta_archival_time_to_unix_timestamp('2014-09-18-09-06')])

Fixing up the action logs `append` was a ~70% speedup!

```
         47807063 function calls (47374260 primitive calls) in 38.861 seconds
```

In [8]:
from processing import parse_tripwise_action_logs_into_trip_log

In [27]:
parse_tripwise_action_logs_into_trip_log(
    [
        parse_message_into_action_log(gtfs_r[0].entity[0], gtfs_r[0].entity[1],
                              mta_archival_time_to_unix_timestamp('2014-09-18-09-01')),
        parse_message_into_action_log(gtfs_r[5].entity[0], gtfs_r[5].entity[1],
                              mta_archival_time_to_unix_timestamp('2014-09-18-09-06')),
        parse_message_into_action_log(gtfs_r[10].entity[0], gtfs_r[10].entity[1],
                                      mta_archival_time_to_unix_timestamp('2014-09-18-09-11'))
    ]
)

,action,latest_information_time,maximum_time,minimum_time,route_id,stop_id,trip_id
0,STOPPED_AT,1411045260,1411045260,1411045260,1,137S,047600_1..S02R
1,STOPPED_OR_SKIPPED,1411045560,1411045560,1411045260,1,138S,047600_1..S02R
2,STOPPED_OR_SKIPPED,1411045560,1411045560,1411045260,1,139S,047600_1..S02R
3,STOPPED_OR_SKIPPED,1411045860,1411045860,1411045560,1,140S,047600_1..S02R


In [28]:
%prun ret2 = parse_feeds_into_trip_logs([gtfs_r[0], gtfs_r[5]], [mta_archival_time_to_unix_timestamp('2014-09-18-09-01'), mta_archival_time_to_unix_timestamp('2014-09-18-09-06')])

Doing the same thing for trip log parsing was a further 50% speedup on top of that.

```
         22965981 function calls (22744790 primitive calls) in 15.865 seconds
```

I can't optimize more than that using `prun` output alone because `cProfiler` is hard to understand. Let's switch tools and see if we can get more performance.

In [31]:
!pip install pyinstrument

    100% |████████████████████████████████| 51kB 1.5MB/s ta 0:00:01
  Running setup.py bdist_wheel for pyinstrument ... - done
  Stored in directory: /home/alex/.cache/pip/wheels/90/30/05/48c02ccd6fb902d6741d9ccda2fb0ddc94ee70d86af4847cb0
Successfully built pyinstrument


In [6]:
import pyinstrument

In [33]:
from pyinstrument import Profiler

profiler = Profiler()
profiler.start()

# code you want to profile
ret2 = parse_feeds_into_trip_logs([gtfs_r[0], gtfs_r[5]], [mta_archival_time_to_unix_timestamp('2014-09-18-09-01'), mta_archival_time_to_unix_timestamp('2014-09-18-09-06')])

profiler.stop()

print(profiler.output_text(unicode=True, color=True))

11.869 parse_feeds_into_trip_logs  processing.py:499
├─ 8.228 parse_message_list_into_action_log  processing.py:46
│  ├─ 7.209 parse_message_into_action_log  processing.py:100
│  │  ├─ 3.673 <listcomp>  processing.py:300
│  │  │  └─ 3.661 __init__  pandas/core/series.py:135
│  │  │     ├─ 1.125 __new__  pandas/indexes/base.py:142
│  │  │     │  ├─ 0.642 __new__  pandas/indexes/base.py:142
│  │  │     │  │  └─ 0.125 _asarray_tuplesafe  pandas/core/common.py:365
│  │  │     │  └─ 0.177 _asarray_tuplesafe  pandas/core/common.py:365
│  │  │     ├─ 0.934 _sanitize_array  pandas/core/series.py:2817
│  │  │     │  └─ 0.805 _try_cast  pandas/core/series.py:2834
│  │  │     │     ├─ 0.556 _possibly_cast_to_datetime  pandas/types/cast.py:765
│  │  │     │     │  └─ 0.364 _possibly_infer_to_datetimelike  pandas/types/cast.py:672
│  │  │     │     └─ 0.174 is_extension_type  pandas/types/common.py:301
│  │  │     ├─ 0.339 __init__  pandas/core/internals.py:4031
│  │  │     │  └─ 0.269 make_block  

This trace shows that most of our time is now spent in the list comprehensions that concatenate the results that we created in the earlier optimizations. So if we can get these running even faster, we can get this library running faster. Time to whip out `numpy` arrays.

In [38]:
parse_message_into_action_log(gtfs_r[0].entity[0], gtfs_r[0].entity[1],
                      mta_archival_time_to_unix_timestamp('2014-09-18-09-01'))

,trip_id,route_id,action,stop_id,time_assigned,information_time
0,047600_1..S02R,1,1411045260,STOPPED_AT,137S,1411044718
1,047600_1..S02R,1,1411045260,EXPECTED_TO_ARRIVE_AT,138S,1411044928
2,047600_1..S02R,1,1411045260,EXPECTED_TO_DEPART_AT,138S,1411044928
3,047600_1..S02R,1,1411045260,EXPECTED_TO_ARRIVE_AT,139S,1411045018
4,047600_1..S02R,1,1411045260,EXPECTED_TO_DEPART_AT,139S,1411045078
5,047600_1..S02R,1,1411045260,EXPECTED_TO_DEPART_AT,139S,1411045078


After.

In [10]:
from processing import parse_message_into_action_log

In [26]:
parse_message_into_action_log(gtfs_r[0].entity[0], gtfs_r[0].entity[1],
                      mta_archival_time_to_unix_timestamp('2014-09-18-09-01'))

,trip_id,route_id,information_time,action,stop_id,time_assigned
0,047600_1..S02R,1,1411045260,STOPPED_AT,137S,1411044718
1,047600_1..S02R,1,1411045260,EXPECTED_TO_ARRIVE_AT,138S,1411044928
2,047600_1..S02R,1,1411045260,EXPECTED_TO_DEPART_AT,138S,1411044928
3,047600_1..S02R,1,1411045260,EXPECTED_TO_ARRIVE_AT,139S,1411045018
4,047600_1..S02R,1,1411045260,EXPECTED_TO_DEPART_AT,139S,1411045078
5,047600_1..S02R,1,1411045260,EXPECTED_TO_ARRIVE_AT,140S,1411045228


In [20]:
from pyinstrument import Profiler

profiler = Profiler()
profiler.start()

# code you want to profile
ret2 = parse_feeds_into_trip_logs([gtfs_r[0], gtfs_r[5]], [mta_archival_time_to_unix_timestamp('2014-09-18-09-01'), mta_archival_time_to_unix_timestamp('2014-09-18-09-06')])

profiler.stop()

print(profiler.output_text(unicode=True, color=True))

5.503 parse_feeds_into_trip_logs  processing.py:491
├─ 2.995 parse_tripwise_action_logs_into_trip_log  processing.py:298
│  ├─ 0.727 iterrows  pandas/core/frame.py:654
│  │  ├─ 0.645 __init__  pandas/core/series.py:135
│  │  │  ├─ 0.352 _sanitize_array  pandas/core/series.py:2817
│  │  │  │  └─ 0.320 _try_cast  pandas/core/series.py:2834
│  │  │  │     ├─ 0.199 _possibly_cast_to_datetime  pandas/types/cast.py:765
│  │  │  │     │  └─ 0.137 _possibly_infer_to_datetimelike  pandas/types/cast.py:672
│  │  │  │     └─ 0.089 is_extension_type  pandas/types/common.py:301
│  │  │  ├─ 0.152 __init__  pandas/core/internals.py:4031
│  │  │  │  └─ 0.119 make_block  pandas/core/internals.py:2649
│  │  │  └─ 0.063 __setattr__  pandas/core/generic.py:2746
│  │  └─ 0.066 values  pandas/core/generic.py:2910
│  │     └─ 0.065 as_matrix  pandas/core/generic.py:2868
│  │        └─ 0.060 _consolidate_inplace  pandas/core/generic.py:2795
│  │           └─ 0.060 _protect_consolidate  pandas/core/generic.py:

That's another ~60% speedup!

In [22]:
%prun ret2 = parse_feeds_into_trip_logs([gtfs_r[0], gtfs_r[5]], [mta_archival_time_to_unix_timestamp('2014-09-18-09-01'), mta_archival_time_to_unix_timestamp('2014-09-18-09-06')])

This now runs in 7 seconds:

         9222335 function calls (9089769 primitive calls) in 7.291 seconds

Let's apply this speedup to the trip logging as well.